In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
from scipy.optimize import minimize

import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import sys

In [2]:
%matplotlib widget

%reload_ext autoreload
%autoreload 2

In [3]:
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
sys.path.append('../../../../nadavp/opt_lev_analysis/lib/')
import BeadDataFile
from discharge_tools import *

In [4]:
from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

In [5]:
import bead_util as bu
# import calib_util as cal
import transfer_func_util as tf
import configuration as config

In [49]:
bandwidth = 1
decimate = 10
drive_freq = 139
fsamp = 5000

### Discharge data

In [77]:
ll = likelihood_analyser.LikelihoodAnalyser()

def func1_scale(i):
    
    bb = step_file_objs[i]      
    freq2 = np.fft.rfftfreq(len(bb.other_data[5]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.other_data[5]))
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)

    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+0.07, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_A': [0, 10000000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}
    
    ## bandpass data
    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    bb.load_other_data()
    xx2 = signal.filtfilt(b, a, bb.pos_data[1])[::decimate]
    ## extract amplitude
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0], m1_tmp.values[2]-phi_tmp2

In [59]:
step_cal_dir = r'/data/old_trap/20211001/bead1/discharge/recharge_20211013/'
max_file = 500

In [78]:
step_cal_files = []
for root, dirnames, filenames in os.walk(step_cal_dir):
    for filename in fnmatch.filter(filenames, '*' + config.extensions['data']):
        if '_fpga.h5' in filename:
            continue
        step_cal_files.append(os.path.join(root, filename))
step_cal_files.sort(key = bu.find_str)
num_calib_files = len(step_cal_files)
print(num_calib_files)

# Do the step calibration

step_file_objs = []
for filname in step_cal_files[:]:
    try:
        df = bu.DataFile();
        df.load(filname);
        df.load_other_data()
        step_file_objs.append(df);
    except:
        print('***')

175


In [79]:
amps2 = []
for i in range(num_calib_files):
    print(i,'/',len(step_cal_files))
    amps2.append(func1_scale(i))

0 / 175
1 / 175
2 / 175
3 / 175
4 / 175
5 / 175
6 / 175
7 / 175
8 / 175
9 / 175
10 / 175
11 / 175
12 / 175
13 / 175
14 / 175
15 / 175
16 / 175
17 / 175
18 / 175
19 / 175
20 / 175
21 / 175
22 / 175
23 / 175
24 / 175
25 / 175
26 / 175
27 / 175
28 / 175
29 / 175
30 / 175
31 / 175
32 / 175
33 / 175
34 / 175
35 / 175
36 / 175
37 / 175
38 / 175
39 / 175
40 / 175
41 / 175
42 / 175
43 / 175
44 / 175
45 / 175
46 / 175
47 / 175
48 / 175
49 / 175
50 / 175
51 / 175
52 / 175
53 / 175
54 / 175
55 / 175
56 / 175
57 / 175
58 / 175
59 / 175
60 / 175
61 / 175
62 / 175
63 / 175
64 / 175
65 / 175
66 / 175
67 / 175
68 / 175
69 / 175
70 / 175
71 / 175
72 / 175
73 / 175
74 / 175
75 / 175
76 / 175
77 / 175
78 / 175
79 / 175
80 / 175
81 / 175
82 / 175
83 / 175
84 / 175
85 / 175
86 / 175
87 / 175
88 / 175
89 / 175
90 / 175
91 / 175
92 / 175
93 / 175
94 / 175
95 / 175
96 / 175
97 / 175
98 / 175
99 / 175
100 / 175
101 / 175
102 / 175
103 / 175
104 / 175
105 / 175
106 / 175
107 / 175
108 / 175
109 / 175
110 / 175


In [80]:
# amps = np.array(amps)
amps2 = np.array(amps2)
# amps1 = np.array(amps1)
_,ax = plt.subplots(figsize=(9.5,4))
# scale = 375

# Perrinf
# ax.scatter(range(len(amps3)), amps3/scaleY, label='fit phase', alpha=0.3)
ax.scatter(range(len(amps2)), amps2[:,0]/scaleY, label='fit phase')
# ax.scatter(range(len(amps)), amps/scaleY, label='fit phase', alpha=0.4)
# ax.scatter(range(len(amps1)), amps1/scale, label='fit phase', alpha=0.8)

ax.set(xlabel='dataset', ylabel='electron charges', ylim=(-0.3,4.5), xlim=(None,None))
ax.grid(axis='y')
ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
# ax.legend()
from matplotlib.ticker import MaxNLocator

ax.yaxis.set_major_locator(MaxNLocator(integer=True))

/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [81]:
# scale = -1*np.abs(np.mean(amps3[200:250]))/2
# scale = -1*np.abs(np.mean(amps[230:240]))/3
# scale = -1*np.abs(np.mean(amps[97:118]))/4
scaleY = np.abs(np.mean(amps2[150:,0]))/4
print("Scales are ", scaleY)
bb = step_file_objs[-1]
drive_freq = 139
bandwidth=5
b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
xx2 = signal.filtfilt(b, a, bb.other_data[5])
voltage = np.std(xx2)*np.sqrt(2)*100
print("voltage: ", voltage)

Scales are  101.058262361
voltage:  24.4577396046


In [82]:
# scaleY = 93.2359
scaleZ = scaleY*(418+627)/(392+406)
scaleX = scaleY*(421+368)/(392+406)
scaleX, scaleY, scaleZ

(99.918507522647417, 101.05826236130879, 132.3382007112377)

### Look at neutrality data

In [26]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 1
decimate = 2
def get_amp(i):
    
    fname = r'/data/old_trap/20211001/bead1/neutrality5/Y_175Vpp_alternate_139Hz_spin_XZ_77kHz_no_delay_no_height_fb/Y_175Vpp_'+str(i)+'.h5'
#     fname = r'/data/old_trap/20211001/bead1/discharge/recharge_20211013/Y_50Vpp_'+str(i)+'.h5'

    drive_freq = 139
    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()
    
    jj = i%2
    freq2 = np.fft.rfftfreq(len(bb.other_data[5+jj]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.other_data[5+jj]))
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)

#     fit_kwargs = {'A': 0.23, 'f': drive_freq, 'phi': 0, 
#                   'error_A': 1, 'error_f': 10, 'error_phi': 0.5, 'errordef': 1,
#                   'limit_A': [0, 1], 'fix_f': True, 'fix_phi': False,
#                   'print_level': 0}

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
#     xx2 = signal.filtfilt(b, a, bb.other_data[5+jj])[::decimate]    
#     m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
#     phi_tmp = m1_tmp.values[2]

#     print(phi_tmp, phi_tmp2, m1_tmp.values[0]*100, np.std(xx2)*np.sqrt(2)*100)
    
    if jj==0:
        phase_diff = 0.12
    else:
        phase_diff = 0.05
            
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+phase_diff, 'sigma': 1100,
                  'error_A': 0.1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma':50,
                  'limit_A': [-100000, 100000], 'limit_sigma':[100,30000], 'fix_phi': True, 'fix_f': True,
                  'print_level': 0}

    xx2 = signal.filtfilt(b, a, bb.pos_data[1])[::decimate]
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    drive_freq = drive_freq*2
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+np.pi/2+phase_diff, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_A': [-100000, 100000], 'fix_f': True, 'fix_phi': True,
                  'print_level': 0}

        
    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.pos_data[1])[::decimate]
    m2_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)

    #     print(m1_tmp.values)
    return m1_tmp.values[0], m2_tmp.values[0], m1_tmp.values[3], m1_tmp.fval

In [285]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
def get_ampZ(i):
    
    fname = r'/data/old_trap/20211001/bead1/neutrality5/Z_150Vpp_alternate_139Hz_spin_XZ_93kHz_delay_6sec_no_height_fb_long/turbombar_powfb_xyzcool_'+str(i)+'.h5'
#     fname = r'/data/old_trap/20211001/bead1/single_elect/rode_20211013/Z_minus_139Hz_50Vpp/Z_50Vpp_'+str(i)+'.h5'

    drive_freq = 139
    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()

    ## define bandpass
    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    
    jj = i%2
    jj=1
    freq2 = np.fft.rfftfreq(len(bb.other_data[1+jj]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.other_data[1+jj]))
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)

#         fit_kwargs = {'A': 0.23, 'f': drive_freq, 'phi': 0, 
#                   'error_A': 1, 'error_f': 10, 'error_phi': 0.5, 'errordef': 1,
#                   'limit_A': [0, 1], 'fix_f': True, 'fix_phi': False,
#                   'print_level': 0}

#     xx2 = signal.filtfilt(b, a, bb.other_data[5+jj])[::decimate]    
#     m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
#     phi_tmp = m1_tmp.values[2]

#     print(phi_tmp, phi_tmp2, m1_tmp.values[0]*100, np.std(xx2)*np.sqrt(2)*100)
    
    if jj==0:
        phase_diff = 4.8
    else:
        phase_diff = 4.8
            
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+phase_diff, 'sigma': 1100,
                  'error_A': 0.1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma':50,
                  'limit_A': [-100000, 100000], 'limit_sigma':[100,3000000], 'fix_phi': True, 'fix_f': True,
                  'print_level': 0}

    xx2 = signal.filtfilt(b, a, bb.pos_data[2])[::decimate]
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    drive_freq = drive_freq*2
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+phase_diff+np.pi/2, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_A': [-1000000, 1000000], 'fix_f': True, 'fix_phi': True,
                  'print_level': 0}

        
    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.pos_data[2])[::decimate]
    m2_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)

    return m1_tmp.values[0], m2_tmp.values[0], m1_tmp.values[3], m1_tmp.fval

In [286]:
i=0
get_ampZ(i)[0]

1.4438342903304147

In [88]:
i = 0
fname = r'/data/old_trap/20211001/bead1/neutrality6/Z_150Vpp_alternate_139Hz_spin_XY_93kHz_200Vpp_delay_3sec_with_height_fb/Z_200Vpp_'+str(i)+'.h5'
bb = bu.DataFile();
bb.load(fname);
bb.load_other_data()
print("voltage: ", voltage, np.mean(bb.other_data[1])*100, np.mean(bb.other_data[2])*100)
drive_freq = 139
bandwidth=2
b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
xx2 = signal.filtfilt(b, a, bb.other_data[1])
voltage_200_1 = np.std(xx2)*np.sqrt(2)*100
xx3 = signal.filtfilt(b, a, bb.other_data[2])
voltage_200_2 = np.std(xx3)*np.sqrt(2)*100
print("voltage: ", voltage_200_1, voltage_200_2)

voltage:  24.0265126779 -0.57213946746 -0.57875944152
voltage:  190.952315144 0.672970840708


In [59]:
voltage, voltage_150_1

(24.026512677896338, 143.17712232598214)

In [60]:
def get_amp_wraper(i):
    try:
        return get_amp(i)
    except:
        return 0,0,0,0
    
def get_amp_wraperZ(i):
    try:
        return get_ampZ(i)
    except:
        return 0,0,0,0

In [287]:
amps_spin_xz_93kHz_z_alternate_150Vpp = np.array(Parallel(n_jobs=40)(delayed(get_amp_wraperZ)(i) for i in tqdm(range((1400)))))

100%|██████████| 1400/1400 [00:26<00:00, 61.58it/s]


In [242]:
i=0; j=2
_,ax = plt.subplots()
dat = amps_spin_93kHz_y_200Vpp_alternate_no[::2]
# dat1 = amps_with_fb_spin_xz_100Vpp_77kHz_y_minus_175Vpp_alternate_no_delay[1::2]/scaleZ*voltage/voltage_150_1
ax.scatter((dat[:,i]), (dat[:,j]))
# ax.scatter(dat[:,i], np.abs(dat[:,1]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [256]:
[i for i,bb in enumerate(amps_spin_93kHz_y_200Vpp_alternate_no) if bb[2]>20000]
# len(amps_spin_93kHz_y_200Vpp_alternate_no_tmp)

[692]

In [257]:
amps_spin_93kHz_y_200Vpp_alternate_no_2 = list(amps_spin_93kHz_y_200Vpp_alternate_no)

In [259]:
amps_spin_93kHz_y_200Vpp_alternate_no_2.pop(692)

array([   -8.91259168,    89.5805347 ,  1331.65569017,    15.38929276])

In [260]:
len(amps_spin_93kHz_y_200Vpp_alternate_no_2)

1998

In [291]:
voltage_150_1, voltage_200_1

(143.17712232598214, 190.95231514383175)

In [302]:
amps_all = list(amps_spin_93kHz_y_200Vpp_alternate_no_2)
# amps_all += list(amps_spin_93kHz_y_150Vpp_alternate_no)
# amps_all = list(amps_with_fb_spin_xy_150Vpp_103kHz_z_200Vpp_alternate_no_delay_redo3)
# amps_all += list(amps_with_fb_spin_xy_150Vpp_103kHz_z_200Vpp_alternate_no_delay_redo4)
amps_all = np.array(amps_all)

In [299]:
amps_all = np.array(amps_spin_xz_93kHz_z_alternate_150Vpp)

In [303]:
etha = 1/scaleZ*327*1.6e-19*voltage
to_epsilon = 1/(2*327*1.6e-19*voltage_200_1*(418)/(418+627))

In [304]:
j = 0
data = amps_all[::2,j]*etha
data2 = amps_all[1::2,j]*etha
data3 = data-0.66*data2

data_1 = np.abs(amps_all[::2,1])*etha
data2_1 = np.abs(amps_all[1::2,1])*etha
data3_1 = data_1+0.66*0.66*data2_1

_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(range(len(data)), data, label='200Vpp-z-plus, xz-spin-50Vpp-93kHz')
ax.scatter(range(len(data2)), data2, label='200Vpp-z-minus, xz-spin-50Vpp-93kHz', alpha=0.6)
# ax.scatter(range(len(data3)), data3, label='200Vpp-z, A parameter', alpha=0.6)

# ax.scatter(range(len(data4)), data4, label='20V-x, xy-spin-30V, w/o height fb')
ax.plot(np.arange(0,1000), np.arange(0,1000)*0, '--k')
ax.set(xlabel='Dataset', ylabel=r'$Force [N]$')
ax.legend()
print('1st harmonic, electrode 1: mean, std: ', np.mean(data)*to_epsilon, np.std(data)/np.sqrt(len(data)))
print('1st harmonic, electrode 2: mean, std: ', np.mean(data2)*to_epsilon, np.std(data2)/np.sqrt(len(data2)))
print('1st harmonic, A parameter: mean, std: ', np.mean(data3)*to_epsilon, np.std(data3)*to_epsilon/np.sqrt(len(data3)))
print()
print('2nd harmonic, electrode 1: mean, std: ', np.mean(data_1)*to_epsilon, np.std(data_1)/np.sqrt(len(data_1)))
print('2nd harmonic, electrode 2: mean, std: ', np.mean(data2_1)*to_epsilon, np.std(data2_1)/np.sqrt(len(data2_1)))
print('2nd harmonic, B parameter: mean, std: ', np.mean(data3_1)*to_epsilon, np.std(data3_1)*to_epsilon/np.sqrt(len(data3_1)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1st harmonic, electrode 1: mean, std:  -0.00605713534169 2.60334823239e-18
1st harmonic, electrode 2: mean, std:  -0.00786211629238 3.37075492191e-18
1st harmonic, A parameter: mean, std:  -0.000868138588718 0.000197903041369

2nd harmonic, electrode 1: mean, std:  0.0331719057307 4.21988102675e-18
2nd harmonic, electrode 2: mean, std:  0.0676557128048 8.59182565274e-18
2nd harmonic, B parameter: mean, std:  0.0626427342285 0.000725855188309


In [310]:
0.0447/0.000677, 0.062/0.00086, 0.00086*0.0447/0.062-0.000677

(66.02658788774002, 72.09302325581396, -5.696774193548397e-05)

In [147]:
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp

n_bins = 21
range_pram = 0.03
_,ax = plt.subplots()
entries, bins, patches = ax.hist(data3*to_epsilon, bins=n_bins, range=(-range_pram, range_pram), label='125Vpp+150Vpp, 93kHz')
# entries3, bins, patches = ax.hist(data3, bins=n_bins, range=(-range_pram, range_pram), label='150Vpp, 77kHz', alpha=0.6)
ax.set(xlabel=r'$\epsilon$')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [148]:
## gaussian fit to the histogram

bins_fit =[(bins[i+1]-bins[i])/2+bins[i] for i in range(len(entries))]

titles = ['', '', '']

for i,ee in enumerate([entries]):
    x = bins_fit
    y= np.array(ee)
    sigma_bin = np.sqrt(ee)
    
    n = len(x)                          #the number of data
    mean = 0                  #note this correction
    sigma = 0.023       #note this correction

    def gaus(x,a,x0,sigma):
        return a*np.exp(-(x-x0)**2/(2*sigma**2))

    popt,pcov = curve_fit(gaus,x,y,p0=[1,mean,sigma])
    ax.plot(x,gaus(x,*popt),'ro:',label='fit')

    print("mean: ", popt[1], " ,std (on the mean): ", np.sqrt(pcov)[1,1], " ,sigma: ", popt[2]/np.sqrt(len(data3)))
    print('\nNeutrality: ', 1/1.26e14*popt[1])

mean:  -0.000781728483835  ,std (on the mean):  0.000201906749607  ,sigma:  -0.000205393744226

Neutrality:  -6.20419431615e-18


/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in sqrt


### Data Analysis

In [323]:
etha = 1/scaleZ*327*1.6e-19*voltage
to_epsilon = 1/(2*327*1.6e-19*voltage_200_1*(418)/(418+627))

In [338]:
data_current = np.array(amps_spin_93kHz_y_200Vpp_alternate_no_2)
z_top = data_current[::2,0]*etha
z_bottom = data_current[1::2,0]*etha

In [325]:
len(data_current), bin_size, bins

(1998, 10, 99)

In [370]:
z_top_means = []
z_bottom_mean = []
z_top_std = []
z_bottom_std = []

bin_size = 10
bins = len(z_top)//bin_size
for i in range(bins):
    z_top_means.append(np.mean(z_top[i*bin_size:(i+1)*bin_size]))
    z_bottom_mean.append(np.mean(z_bottom[i*bin_size:(i+1)*bin_size]))
    z_top_std.append(np.std(z_top[i*bin_size:(i+1)*bin_size]))
    z_bottom_std.append(np.std(z_bottom[i*bin_size:(i+1)*bin_size]))
z_top_means = np.array(z_top_means)
z_top_std = np.array(z_top_std)
z_bottom_mean = np.array(z_bottom_mean)
z_bottom_std = np.array(z_bottom_std)

In [371]:
_,ax = plt.subplots(figsize=(9.4,4))
iteration = np.arange(len(z_top_means)) 
ax.plot(iteration, iteration*0, 'r--')
ax.errorbar(iteration, z_top_means, yerr=z_top_std/np.sqrt(10), fmt='o')
ax.errorbar(iteration, z_bottom_mean*0.66, yerr=z_bottom_std/np.sqrt(10), fmt='o')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ErrorbarContainer object of 3 artists>

In [376]:
_,ax = plt.subplots(figsize=(9.4,4))
iteration = np.arange(len(z_top_means))*10*bin_size
ax.plot(iteration, iteration*0, 'r--')
top_err = z_top_std/np.sqrt(10)
bottom_err = z_bottom_std/np.sqrt(10)
A = z_top_means-z_bottom_mean*0.66
A_err = np.sqrt(top_err**2+top_err**2*0.66*0.66)
ax.errorbar(iteration, z_top_means, yerr=top_err, fmt='o', label='top electrode')
ax.errorbar(iteration, z_bottom_mean*0.66, yerr=bottom_err, fmt='o', label='bottom electrode')
ax.errorbar(iteration, A, yerr=A_err, fmt='o', label='A parameter')
ax.legend()
ax.set(xlabel='Measuring time [s]', ylabel='Measured force [N]')
# ax.errorbar(iteration, (z_top_means-z_bottom_mean*0.66)*to_epsilon, yerr=z_bottom_std/np.sqrt(10)*to_epsilon, fmt='o')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, 'Measuring time [s]'), Text(0, 0.5, 'Measured force [N]')]

In [377]:
A_epsion = A*to_epsilon
A_err_epsilon = A_err*to_epsilon
def calculate_pl(force_):
    chi2 = np.power((A_epsion-force_)/A_err_epsilon, 2)
    return np.sum(chi2)

In [378]:
_,ax = plt.subplots()
pl_range = np.arange(-1e-3,1e-3,1e-4)
pl = np.array([calculate_pl(pl_range_) for pl_range_ in pl_range])
ax.scatter(pl_range*1000, pl)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [329]:
_,ax = plt.subplots()
ax.scatter(z_top_means, z_bottom_mean)
ax.set(xlabel='Force Top electroe [N]', ylabel='Force Bottom electroe [N]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, 'Force Top electroe [N]'),
 Text(0, 0.5, 'Force Bottom electroe [N]')]